In [1]:
from scipy.stats import hypergeom
import numpy as np
from gaste_test import get_pval_comb
from tqdm import tqdm

In [2]:
def simu_null_model(range_margins_A, range_margins_B):
    N = 100
    nb_simu=1000 
    nb_strat = 100
    all_comb_under = []
    all_comb_over = []
    min_A, max_A = range_margins_A
    min_B, max_B = range_margins_B
    for _ in tqdm(range(nb_simu)):
        pval_over, pval_under, params = [], [], []
        for _ in range(nb_strat):
            nA = np.random.randint(min_A, max_A)
            nB = np.random.randint(min_B, max_B)
            occ_A = np.random.choice(range(N), nA, replace=False)
            occ_B = np.random.choice(range(N), nB, replace=False)
            occ_A_B = len(set(occ_A) & set(occ_B))
            params.append((N, nA, nB, occ_A_B))
            pval_under.append(hypergeom.cdf(
                occ_A_B, N, nA, nB
            ))
            pval_over.append(hypergeom.sf(
                occ_A_B-1, N, nA, nB
            ))

        comb_pval_under = get_pval_comb(params, pval_under,"under", tau=1, moment=2)
        comb_pval_over = get_pval_comb(params, pval_over,"over", tau=1, moment=2)
        all_comb_under.append(comb_pval_under)
        all_comb_over.append(comb_pval_over)
    all_comb_under = np.array(all_comb_under)
    all_comb_over = np.array(all_comb_over)
    print(np.mean(all_comb_under<=0.05), np.mean(all_comb_over<=0.05))
    return all_comb_under, all_comb_over

# Example usage
all_comb_under_mid, all_comb_over_mid = simu_null_model((10, 90), (10, 90))
all_comb_under_low, all_comb_over_low= simu_null_model((2, 20), (2,20))
all_comb_under_high, all_comb_over_high = simu_null_model((70, 90), (70, 90)) 

100%|██████████| 1000/1000 [10:26<00:00,  1.60it/s]


0.047 0.054


100%|██████████| 1000/1000 [06:12<00:00,  2.69it/s]


0.044 0.051


100%|██████████| 1000/1000 [08:34<00:00,  1.94it/s]

0.048 0.069


In [7]:
from statsmodels.stats.multitest import multipletests
def get_fdr_corrected_pvals(pvals):
    _, corrected_pvals, _, _ = multipletests(pvals, method='fdr_bh')
    return corrected_pvals
corrected_over_mid = get_fdr_corrected_pvals(all_comb_under_mid)
np.mean(corrected_over_mid<=0.05)

np.float64(0.0)